In [12]:
import sys, os
print(sys.version)
print(os.environ['HOME'])
java_path = '/home/jovyan/conda'
os.environ['JAVA_HOME'] = java_path
print(os.environ.get('JAVA_HOME'))

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]
/home/jovyan
/home/jovyan/conda


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Exception: Java gateway process exited before sending its port number

In [22]:
from pyspark.dbutils import DBUtils

dbutils = DBUtils(spark.sparkContext)
dbutils.fs.ls("/mnt")

[FileInfo(path='dbfs:/mnt/AWStest/', name='AWStest/', size=0),
 FileInfo(path='dbfs:/mnt/WoS/', name='WoS/', size=0),
 FileInfo(path='dbfs:/mnt/test/', name='test/', size=0),
 FileInfo(path='dbfs:/mnt/test2/', name='test2/', size=0),
 FileInfo(path='dbfs:/mnt/test3/', name='test3/', size=0),
 FileInfo(path='dbfs:/mnt/test7/', name='test7/', size=0)]

In [2]:
import pandas as pd
ego = pd.read_csv('../../packages/issi_data_package/output_files/data/774e7eb6-6ac7-4dd5-9339-531b746cb8ec.csv')
degree1 = pd.read_csv('../../packages/issi_data_package/output_files/data/774e7eb6-6ac7-4dd5-9339-531b746cb8ec_nodes.csv')
#display(ego,degree1)

In [3]:
edges = pd.read_csv('../../packages/issi_data_package/output_files/data/a71d68fa-a479-47b6-8e55-ce8f12eacb56_edges.csv')
citations = pd.merge(edges, ego, left_on=['To'], right_on=['paper_id'])
citations.count()
#edges.count()

From                    105242
To                      105242
paper_id                105242
year                    105242
original_title          105242
authors_display_name    105242
journal_display_name    105242
paper_abstract          103151
dtype: int64

In [4]:
citations0 = citations[['From','To']]
citations0.columns = ['focal', 'connector']
citations2 = pd.merge(citations0, citations, left_on=['connector'], right_on=['To'])
citations2.count()

focal                   4027234
connector               4027234
From                    4027234
To                      4027234
paper_id                4027234
year                    4027234
original_title          4027234
authors_display_name    4027234
journal_display_name    4027234
paper_abstract          3964481
dtype: int64

In [5]:
citations00 = citations0
citations00.columns = ['cited2','LE']
citations3 = pd.merge(citations2, citations00, left_on=['focal','To'], right_on=['cited2','LE'])
citations3.count()

focal                   6657184
connector               6657184
From                    6657184
To                      6657184
paper_id                6657184
year                    6657184
original_title          6657184
authors_display_name    6657184
journal_display_name    6657184
paper_abstract          6562803
cited2                  6657184
LE                      6657184
dtype: int64

In [6]:
# direct citations
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict

paper_citing = defaultdict(list)

citingFile = open("../../packages/issi_data_package/output_files/data/a71d68fa-a479-47b6-8e55-ce8f12eacb56_edges.csv", "r")
citingFile.readline()
for line in citingFile:
    line = line.strip().replace('"', '').split(",")
    paper_citing[int(line[1])].append(int(line[0]))

In [11]:
# indirect citations
count = 0
total = len(degree1)
indirect_citation = defaultdict(list)
paperList = ego['paper_id'].tolist()

for paper in paperList:
    count += 1
    if count % 1000 == 0:
        print("Processed " + str(float(count) / float(total)) + " of total focal papers...\n")
    for citing_paper_1 in paper_citing[paper]:
        for citing_paper_2 in paper_citing[paper]:
            if citing_paper_1 in paper_citing[citing_paper_2]:
                temp = []
                temp.append(citing_paper_1)
                temp.append(citing_paper_2)
                indirect_citation[paper].append(temp)

Processed 0.025060772373004536 of total focal papers...

Processed 0.05012154474600907 of total focal papers...

Processed 0.0751823171190136 of total focal papers...

Processed 0.10024308949201814 of total focal papers...

Processed 0.12530386186502268 of total focal papers...

Processed 0.1503646342380272 of total focal papers...



In [30]:
import pyspark.sql.functions as sql
papers = spark.read.format("parquet").load("/mnt/test/graph/2019-1-11-PNAS/papers/*.parquet")
papers.describe().filter(sql.col("summary") == "count").show(20,False)

+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+
|summary|_c0   |_c1   |_c2   |_c3   |_c4   |_c5   |_c6|_c7   |_c8   |_c9   |_c10  |_c11|_c12|_c13  |_c14  |_c15  |_c16  |_c17  |_c18  |_c19  |_c20  |_c21  |
+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+
|count  |138953|138953|135508|138953|138953|138953|25 |138953|138953|135368|138953|0   |0   |135531|135099|133690|127181|138953|138953|138953|138953|138953|
+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+



In [11]:
help('modules')


Please wait a moment while I gather a list of all available modules...



/home/jovyan/conda/dbconnect/lib/python3.5/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/home/jovyan/conda/dbconnect/lib/python3.5/pkgutil.py:104: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, name+'.', onerror)


CDROM               autoreload          jsonschema          select
DLFCN               backcall            jupyter             selectors
IN                  base64              jupyter_client      send2trash
IPython             bdb                 jupyter_core        setuptools
PyQt5               binascii            keyword             shelve
TYPES               binhex              kiwisolver          shlex
__future__          bisect              lib2to3             shutil
_ast                bleach              linecache           signal
_bisect             builtins            locale              simplegeneric
_bootlocale         bz2                 logging             sip
_bz2                cProfile            lzma                sipconfig
_codecs             calendar            macpath             sipdistutils
_codecs_cn          certifi             macurl2path         site
_codecs_hk          cgi                 mailbox             six
_codecs_iso2022     cgitb               mail